In [ ]:
import sys
import numpy as np
import cv2
import cv2.ml
from time import sleep
from datetime import datetime
import os
import numpy as np
from matplotlib import pyplot as plt
from pandas import read_csv, read_excel, DataFrame
from sklearn.mixture import GaussianMixture
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandmForestClassifier
from sklearn import svm
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import logging
logging.basicConfig(level=logging.INFO)
p = os.path.join('../../','raw','CookIRCamET','Images','CookTests','HiRes')
p2 = os.path.join('../../','work','CookIRCamET','Images','CookTests','HiRes')
n_feat = 4
n_components = 4

In [ ]:
def localSD(mat, n):    
    mat=np.float32(mat)
    mu = cv2.blur(mat,(n,n))
    mdiff=mu-mat
    mat2=cv2.blur(np.float64(mdiff*mdiff),(n,n))
    sd = np.float32(cv2.sqrt(mat2))
    
    return sd

In [ ]:
f_imgs=[]
test_imgs=[]
train_imgs=[]
for f in os.listdir(p):
    if 'bgr' in f:
        f_imgs = np.append(f_imgs,f)
        bgr = cv2.imread(os.path.join(p,f),cv2.IMREAD_UNCHANGED)
        f_labels = f.split('_bgr')[0]+'_labels.tif'
        labels = cv2.imread(os.path.join(p,f_labels),cv2.IMREAD_UNCHANGED)
        lab = cv2.cvtColor(bgr, cv2.COLOR_BGR2LAB)
        img = cv2.cvtColor(bgr, cv2.COLOR_BGR2GRAY)
        l,a,bb = cv2.split(lab)
        sd = localSD(l, 451)
        img_size = l.shape
        bb = bb.ravel()
        a = a.ravel()
        l = l.ravel()
        sd = sd.ravel()
        feat = np.vstack((l.T,a.T,bb.T,sd.T)).T
        plt.subplot(211)
        plt.imshow(bgr)
        plt.subplot(212)
        plt.imshow(labels)
        plt.show()
        #labels = np.sum(np.vstack((soil.ravel().T, residue.ravel().T*2, shadow.ravel().T*3, vegetation.ravel().T*4)).T,axis=1)
        labels = labels.ravel()
        if f.split('_')[0]=='20220509152045' or f.split('_')[0]=='20220509152630':
            train_imgs.append({'bgr':bgr,'feats':feat,'labels':labels})
        else:
            test_imgs.append({'bgr':bgr,'feats':feat,'labels':labels})

In [ ]:
train_feats = []
train_labels = []
for sample in train_imgs:
    train_feats.append(sample['feats'])
    train_labels.append(sample['labels'])

In [ ]:
test_feats = []
test_labels = []
for sample in test_imgs:
    test_feats.append(sample['feats'])
    test_labels.append(sample['labels'])

In [ ]:
train_feats = np.array(train_feats).reshape((-1,n_feat)).astype(np.float32)
train_labels = np.array(train_labels).reshape((-1,1)).astype(np.int32)
test_feats = np.array(test_feats).reshape((-1,n_feat)).astype(np.float32)
test_labels = np.array(test_labels).reshape((-1,1)).astype(np.int32)

In [ ]:
parameters= { 
    'n_estimators': [100,200,400],
    'max_features': ['auto', 'sqrt'],
    'max_depth' : [4,5,6,7,8],
    'criterion' :['gini', 'entropy']}
rfc = RandomForestClassifier()
clf_rfc = GridSearchCV(rfc, parameters,n_jobs=-1)
clf_rfc.fit(train_feats,train_labels)

In [ ]:
parameters = {'kernel':('linear', 'rbf'), 'C':np.logspace(-3,3,7), 'gamma':np.logspace(-4,0,5)}
svc = svm.SVC()
clf_svc = GridSearchCV(svc, parameters,n_jobs=-1)
clf_svc.fit(train_feats, train_labels)

In [ ]:
def cornfusion(obs,pred,nclass):
    M = np.zeros((nclass,nclass))
    for i in range(obs.shape[0]):
        o = obs[i][0]
        p = pred[i][0]
        M[o,p] = M[o,p]+1
    return M

In [ ]:
M_svm_rbf = cornfusion(test_labels,pred_svm_rbf,n_components)

M_svm_lin = cornfusion(test_labels,pred_svm_lin,n_components)

M_svm_pol = cornfusion(test_labels,pred_svm_pol,n_components)

M_nb = cornfusion(test_labels,pred_nb,n_components)

M_rt = cornfusion(test_labels,pred_rt,n_components)

M_em = cornfusion(test_labels,pred_em,n_components)

M_rt = M_rt/np.sum(M_rt,axis=1).reshape((-1,1))

M_nb = M_nb/np.sum(M_nb,axis=1).reshape((-1,1))

M_svm_rbf = M_svm_rbf/np.sum(M_svm_rbf,axis=1).reshape((-1,1))

M_svm_lin = M_svm_lin/np.sum(M_svm_lin,axis=1).reshape((-1,1))

M_svm_pol = M_svm_pol/np.sum(M_svm_pol,axis=1).reshape((-1,1))

M_em = M_em/np.sum(M_em,axis=1).reshape((-1,1))